In [42]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
from dask.delayed import delayed
from zipfile import ZipFile
import hvplot.pandas
import hvplot.dask
import random
from datetime import date, datetime, timedelta, time
from tqdm import tqdm

In [2]:
def load_zipped_csv(file_path):
    zipped_file = [delayed(pd.read_csv)(file_path)]
    return dd.from_delayed(zipped_file)

In [3]:
def add_total_number_of_days(df):
    df_to_join = df.groupby("user_id").days_since_prior_order.sum().reset_index().rename(columns={"days_since_prior_order":"total_days_on_platform"})
    df_to_join['user_id'] = df_to_join['user_id'].astype(int)
    df = pd.merge(df, df_to_join, on=["user_id"])
    return df

In [4]:
def add_initial_date(df):
    pass

## Load the data

In [5]:
df_order_products = load_zipped_csv("../data/order_products__prior.csv.zip")
df_orders = load_zipped_csv("../data/orders.csv.zip")
df_products = load_zipped_csv("../data/products.csv.zip")
df_aisles = load_zipped_csv("../data/aisles.csv.zip")
df_departments = load_zipped_csv("../data/departments.csv.zip")

In [6]:
df_orders = df_orders[df_orders["eval_set"] == "prior"]
# df_orders = df_orders[df_orders["user_id"].isin([4, 5, 6, 7])]
df_orders['order_id'] = df_orders['order_id'].astype(int)


df_order_products = df_order_products.groupby("order_id")["product_id"].apply(list).compute().to_frame().reset_index()
df_order_products['order_id'] = df_order_products['order_id'].astype(int)

<ipython-input-6-c4d96f22324e>:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  df_order_products = df_order_products.groupby("order_id")["product_id"].apply(list).compute().to_frame().reset_index()


In [7]:
df = df_orders.merge(df_order_products, on=["order_id"]).compute()

# df = df.join(df_products, on='product_id', rsuffix="_")
# df = df.join(df_aisles, on='aisle_id', rsuffix="_")
# df = df.join(df_departments, on='department_id', rsuffix="_")

In [8]:
df = add_total_number_of_days(df)

## Joined dataset metadata

In [9]:
df.head(5)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,total_days_on_platform
0,2539329,1,prior,1,2,8,NaN,"[196, 14084, 12427, 26088, 26405]",176.0
1,2398795,1,prior,2,3,7,15.0,"[196, 10258, 12427, 13176, 26088, 13032]",176.0
2,473747,1,prior,3,3,12,21.0,"[196, 12427, 10258, 25133, 30450]",176.0
3,2254736,1,prior,4,4,7,29.0,"[196, 12427, 10258, 25133, 26405]",176.0
4,431534,1,prior,5,4,15,28.0,"[196, 12427, 10258, 25133, 10326, 17122, 41787...",176.0


In [10]:
n_rows = len(df)
n_rows

3214874

In [11]:
for col in df:
    try:
        print(col, len(df[col].unique()))
    except:
        pass

order_id 3214874
user_id 206209
eval_set 1
order_number 99
order_dow 7
order_hour_of_day 24
days_since_prior_order 32
total_days_on_platform 366


## Plots on orders

In [12]:
df_order_hour = df.groupby("order_hour_of_day").order_id.count().reset_index()
df_order_hour['percentage of orders'] = df_order_hour['order_id']/n_rows
df_order_hour.hvplot.bar(x="order_hour_of_day", y="percentage of orders", title="Percentage of orders per hour")

:Bars   [order_hour_of_day]   (percentage of orders)

In [13]:
df_ = df.groupby("user_id").days_since_prior_order.mean().reset_index()
df_.hvplot.hist("days_since_prior_order" , title="Average number of days between orders")

:Histogram   [days_since_prior_order]   (days_since_prior_order_count)

In [14]:
df.hvplot.hist("order_dow" , title="Day of the week order")

:Histogram   [order_dow]   (order_dow_count)

In [15]:
df_ = df.groupby("user_id").order_number.count().reset_index()
df_.hvplot.hist("order_number" , title="Average number of orders per clients", bins=50)

:Histogram   [order_number]   (order_number_count)

In [16]:
df_ = df.groupby("user_id").days_since_prior_order.sum().reset_index()
df_.hvplot.hist("days_since_prior_order" , title="Number of days since clients uses Instacart", bins=50)

:Histogram   [days_since_prior_order]   (days_since_prior_order_count)

In [54]:
test = df[df['days_since_prior_order'].isna()]
dates = []

hour_of_day = df["order_hour_of_day"].to_list()
delta_days = df["days_since_prior_order"].to_list()
total_days = df["total_days_on_platform"].to_list()

for day, total_day in tqdm(zip(delta_days, total_days), total=len(df), position=0, leave=True):
    if(np.isnan(day)):
        dates.append(date.today() - timedelta(days=total_day-2))
    else:
         dates.append(dates[-1] + timedelta(days=day))

print(len(hour_of_day), len(dates))
dates_hour = []
for hour, date in tqdm(zip(hour_of_day, dates), total=len(df), position=0, leave=True):
    dates_hour.append(datetime.combine(date, time(hour, 0)))

df["date"] = dates_hour

100%|██████████| 3214874/3214874 [00:05<00:00, 567939.27it/s]


3214874 3214874


100%|██████████| 3214874/3214874 [00:02<00:00, 1554187.52it/s]


In [52]:
df

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,total_days_on_platform,date,date_hour
0,2539329,1,prior,1,2,8,NaN,"[196, 14084, 12427, 26088, 26405]",176.0,2023-04-07 08:00:00,2023-04-07 08:00:00
1,2398795,1,prior,2,3,7,15.0,"[196, 10258, 12427, 13176, 26088, 13032]",176.0,2023-04-22 07:00:00,2023-04-07 07:00:00
2,473747,1,prior,3,3,12,21.0,"[196, 12427, 10258, 25133, 30450]",176.0,2023-05-13 12:00:00,2023-04-07 12:00:00
3,2254736,1,prior,4,4,7,29.0,"[196, 12427, 10258, 25133, 26405]",176.0,2023-06-11 07:00:00,2023-04-07 07:00:00
4,431534,1,prior,5,4,15,28.0,"[196, 12427, 10258, 25133, 10326, 17122, 41787...",176.0,2023-07-09 15:00:00,2023-04-07 15:00:00
...,...,...,...,...,...,...,...,...,...,...,...
3214869,2558525,206209,prior,9,4,15,22.0,"[38167, 23892, 6846]",210.0,2023-07-08 15:00:00,2023-04-07 15:00:00
3214870,2266710,206209,prior,10,5,18,29.0,"[9405, 6846, 15700, 26503, 41213, 23800, 24852...",210.0,2023-08-06 18:00:00,2023-04-07 18:00:00
3214871,1854736,206209,prior,11,4,10,30.0,"[6846, 9405, 41213, 24852, 19348, 15700, 40310...",210.0,2023-09-05 10:00:00,2023-04-07 10:00:00
3214872,626363,206209,prior,12,1,12,18.0,"[6846, 24852, 9405, 19348, 41213, 15700, 38167...",210.0,2023-09-23 12:00:00,2023-04-07 12:00:00


In [57]:
df_ = df.groupby("date").order_number.count().reset_index()
df_["date"] = pd.to_datetime(df_["date"])
df_ = df_[(df_["date"].dt.date < date.today()) & (df_["date"].dt.date > date.today() - timedelta(days=120))]
df_.hvplot(y="order_number", x="date", title="Number of orders in the last 120 days")

:Curve   [date]   (order_number)

In [59]:
df_ = df.groupby("date").order_number.count().reset_index()
df_["date"] = pd.to_datetime(df_["date"])
df_ = df_.resample("1d", on="date").sum().fillna(0).rolling(window=5, min_periods=1).mean().reset_index()
df_ = df_[(df_["date"].dt.date < date.today()) & (df_["date"].dt.date > date.today() - timedelta(days=8))]
df_["date"] = df_["date"].astype(str)
df_.hvplot.bar(y="order_number", x="date", title="Last 7 days average")

:Bars   [date]   (order_number)

# Plots on products

In [21]:
l = df["product_id"].to_list()
l = [item for sublist in tqdm(l) for item in sublist]

100%|██████████| 3214874/3214874 [00:02<00:00, 1074499.95it/s]


In [30]:
df_product_occurrence = pd.DataFrame(l, columns=["product_id"])
df_product_occurrence = df_product_occurrence.value_counts().reset_index()
df_product_occurrence = df_product_occurrence.rename(columns={0:"count"})
df_product_occurrence = df_product_occurrence.merge(df_products.compute(), on=["product_id"])
df_product_occurrence = df_product_occurrence.merge(df_aisles.compute(), on=["aisle_id"])
df_product_occurrence = df_product_occurrence.merge(df_departments.compute(), on=["department_id"])
df_product_occurrence

,product_id,count,product_name,aisle_id,department_id,aisle,department
0,24852,472565,Banana,24,4,fresh fruits,produce
1,13176,379450,Bag of Organic Bananas,24,4,fresh fruits,produce
2,21137,264683,Organic Strawberries,24,4,fresh fruits,produce
3,47209,213584,Organic Hass Avocado,24,4,fresh fruits,produce
4,47766,176815,Organic Avocado,24,4,fresh fruits,produce
...,...,...,...,...,...,...,...
49672,17113,2,Nutrish Naturally Delish Chicken Paw Pie Wet D...,40,8,dog food care,pets
49673,38931,2,Select Roasted Lamb with Vegetables & Barley D...,40,8,dog food care,pets
49674,49487,1,Vitality Chicken Breasts with Flaxseed & Vitam...,40,8,dog food care,pets
49675,32464,1,Flavor Snacks,40,8,dog food care,pets


In [31]:
df_product_occurrence[0:5].hvplot.bar(y="count", x="product_name", title="Top 5 products")

:Bars   [product_name]   (count)

In [32]:
df_top_aisles = pd.DataFrame(df_product_occurrence.groupby("aisle")["count"].sum()).reset_index()
df_top_aisles = df_top_aisles.sort_values("count", ascending=False)
df_top_aisles[0:5].hvplot.bar(y="count", x="aisle", title="Top 5 aisles")

:Bars   [aisle]   (count)

In [33]:
df_top_departments = pd.DataFrame(df_product_occurrence.groupby("department")["count"].sum()).reset_index()
df_top_departments = df_top_departments.sort_values("count", ascending=False)
df_top_departments[0:5].hvplot.bar(y="count", x="department", title="Top 5 departments")

:Bars   [department]   (count)